30 points task.
Deadline until 21.05. After 21.05 it will be 15 points task :)

### IN CLASS:
1. Prepare DL env
    * Install conda(Miniconda)/virtualenv
    * Install pandas, numpy, sklearn, matplotlib, tensorflow, Keras, etc
2. Run example (google keras mnist mlp example)
3. Encapsulate building of MLP in function
    * You need encapsulate only BUILDING and COMPILATION (not training)
    * Parameters to encapsulate: amount of layers, amount of units at each layer, activation function, loss function, optimizer.
    * Bonus: regularization (l1,l2,dropout)
4. Train MLP(read this tutorial properly, example) with hyperparameters search on train data. Evaluate model on the test data
    * Use accuracy score as main evaluation metric
    * Test 2,3,4-layer MLPs with different activation functions and optimizers (simple docs can be found at keras.io)
    * For hyperparameters search you can use:
        * GridSearch/RandomSearch from sklearn with KerasClassifier wrapper from keras
        * ParameterGrid from sklearn
        * Just ‘for’ (:

### HOMETASK (Denoising):
5. Add random normal noise to train and test data. Then, repeat step #3 on noisy data
6. Train Simple (Non convolutional) Denoising Autoencoder (read this tutorial properly) and reconstruct data from data with noise. Repeat step #3 on reconstructed data
7. Compare all results using table and plot. Be creative

_TIPS:_
*If you have any errors with tf or keras try: tensorflow==1.8.0, keras==2.2.0*


In [11]:
from __future__ import print_function
import keras.datasets.mnist as mnist
import keras
from keras.layers.core import Dense, Dropout
from keras.utils import np_utils
from keras.models import Sequential
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid
from keras.regularizers import l2,l1
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [12]:
# importing the dataset :D
(X_train, y_train) , (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000,784)
X_train[0].shape

(784,)

In [66]:
class MyModel:
    def __init__(self, layers_activation, batch_size=128, num_classes=10, epochs=10):
        self.layers_activation = layers_activation
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.epochs = epochs
        self.__load_data__()
        self.model = Sequential()
    
    def __load_data__(self):
        (self.x_train, self.y_train) , (self.x_test, self.y_test) = mnist.load_data()
        self.x_train = self.x_train.reshape(60000, 784)
        self.x_test = self.x_test.reshape(10000, 784)
        self.x_train = self.x_train.astype('float32')
        self.x_test = self.x_test.astype('float32')
        self.x_train /= 255
        self.x_test /= 255
        self.y_train = keras.utils.to_categorical(self.y_train, self.num_classes)
        self.y_test = keras.utils.to_categorical(self.y_test, self.num_classes)
        print('Data Loaded with Sample sizes')
        print('Train Sample Size: {}  '.format(self.x_train.shape[0]))
        print('Test Sample Size: {}'.format(self.x_test.shape[0]))
        
    def build_model(self):
        if self.layers_activation:
            for i in range(len(self.layers_activation['layers'])):  
                print(self.layers_activation['regularization'])
                if 'regularization' in self.layers_activation:
                    regularization = l2(self.layers_activation['regularization'][1]) if self.layers_activation['regularization'][0] == 'l2' else l1(self.layers_activation['regularization'][1])
                else:
                    regularization = None
                
                if i==0:
                    # If First layer have to input size otherwise let it go
                    self.model.add(Dense(self.layers_activation['layers'][i], activation=self.layers_activation['activation'], input_shape=self.x_train[0].shape, kernel_regularizer=regularization))
                else:
                    self.model.add(Dense(self.layers_activation['layers'][i], activation=self.layers_activation['activation'], kernel_regularizer=regularization))
            
                if 'dropout' in self.layers_activation:
                    self.model.add(Dropout(self.layers_activation['dropout']))

        else:
            print('Invalid Layer and Mentions: Update the layer activation dropout parameter with the list of dictionary of keys of size, activation and dropout')
        
        #Adding the final Output Layer
        self.model.add(Dense(self.num_classes, activation='softmax'))
        self.model.summary()
        self.model.compile(loss='categorical_crossentropy', optimizer=self.layers_activation['optimizer'],  metrics=['accuracy'])
        
    
    def train_model(self):
        return self.model.fit(self.x_train, self.y_train, batch_size=self.batch_size, epochs=self.epochs, verbose=2, validation_data=(self.x_test, self.y_test))      
    
    def calculate_accuracy(self):
        score = self.model.evaluate(self.x_test, self.y_test, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])
        return score

    def save(self,filename):
        self.model.save(os.path.join(os.getcwd(), filename))
        
        
        

In [67]:
#Structure of my Model
layer_activation = { 'layers': [[512, 512], [512, 512, 512]], 
                     'activation': ['relu', 'elu'],
                     'optimizer' : ['adam'],
                     'dropout' : [0.2],
                     'regularization' : [ ('l2', 0.001) ]
                   }

In [68]:
combinations = list(ParameterGrid(layer_activation))

In [69]:
combinations

[{'activation': 'relu',
  'dropout': 0.2,
  'layers': [512, 512],
  'optimizer': 'adam',
  'regularization': ('l2', 0.001)},
 {'activation': 'relu',
  'dropout': 0.2,
  'layers': [512, 512, 512],
  'optimizer': 'adam',
  'regularization': ('l2', 0.001)},
 {'activation': 'elu',
  'dropout': 0.2,
  'layers': [512, 512],
  'optimizer': 'adam',
  'regularization': ('l2', 0.001)},
 {'activation': 'elu',
  'dropout': 0.2,
  'layers': [512, 512, 512],
  'optimizer': 'adam',
  'regularization': ('l2', 0.001)}]

In [70]:
history = []
i = 0
for layer_activation in combinations:
    model = MyModel(layer_activation, epochs=5)
    model.build_model()
    history.append(model.train_model())
    model.calculate_accuracy()
    i+=1
    model.save('model{}.h5'.format(i))

Data Loaded with Sample sizes
Train Sample Size: 60000  
Test Sample Size: 10000
('l2', 0.001)
('l2', 0.001)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, val

KeyboardInterrupt: 